# PF Expense Projections: Q3-Q4 '23
#### Adrafül Labs

**Notes:**

Expenses will be projected using a combination of:
- COGS(units sold), which can be found in `sales.ipynb`
- Marketing Spend (imported)
- Shipping loss

Charts will be produced using the plotly charting library.

All future projections are estimates.

_________

In [13]:
# imports
import pandas as pd
import numpy as np
import plotly.graph_objects as go

We will first bring in all local data that will be used to forecast projected expenses. We will break down the expenses into the below categories:

In [14]:
projected_expenses_df = pd.DataFrame(columns=['date', 'COGS', 'marketing', 'sg&a', 'shipping_loss', 'total'])

In [15]:
# import sales data
sales_df = pd.read_excel('../../data/outputs/projected_daily_sales.xlsx')
sales_df.drop('Unnamed: 0', axis=1, inplace=True)

# add orders
AOV = 38.44194795783551
PPU = 3.99
sales_df['orders'] = sales_df['total_gross_sales'] / AOV
sales_df['orders'] = sales_df['orders'].astype(int)
sales_df['units_sold'] = sales_df['total_gross_sales'] / PPU
sales_df['units_sold'] = sales_df['units_sold'].astype(int)
projected_expenses_df['date'] = sales_df['date']

# CPU taken from local data (calculated with production calculator)
cpu_df = pd.read_excel('../../data/imports/excel/projected_cpu.xlsx')

**COGS**

We project COGS roughly as a function of the CPU on that inventory times the number of units sold on that day.

In [16]:
projected_expenses_df['COGS'] = sales_df['units_sold'] * cpu_df['projected_cpu']
projected_expenses_df.head()

,date,COGS,marketing,sg&a,shipping_loss,total
0,2023-08-01,889.44,NaN,NaN,NaN,NaN
1,2023-08-02,120.36,NaN,NaN,NaN,NaN
2,2023-08-03,658.92,NaN,NaN,NaN,NaN
3,2023-08-04,89.76,NaN,NaN,NaN,NaN
4,2023-08-05,75.48,NaN,NaN,NaN,NaN


**Shipping Losses**

Shipping losses are calculated using the assumption of average shipping loss per unit (found in the sales_analysis notebook). We have no reason to forecast any improvements on shipping PPU from our 3PL to the customer.

In [17]:
# Taken from sales.ipynb
# Average shipping loss: -1.6242919071679873
ASL = 1.6242919071679873
projected_expenses_df['shipping_loss'] = sales_df['orders'] * ASL
projected_expenses_df.head()

,date,COGS,marketing,sg&a,shipping_loss,total
0,2023-08-01,889.44,NaN,NaN,73.093136,NaN
1,2023-08-02,120.36,NaN,NaN,9.745751,NaN
2,2023-08-03,658.92,NaN,NaN,53.601633,NaN
3,2023-08-04,89.76,NaN,NaN,6.497168,NaN
4,2023-08-05,75.48,NaN,NaN,4.872876,NaN


**Marketing Spend**

Our daily marketing spend can be found in the data directory.

In [18]:
# Pre-calculated
ms_df = pd.read_excel('../../data/imports/excel/marketing_projected_expenses.xlsx', index_col=0)
projected_expenses_df['marketing'] = list(ms_df['total_marketing_spend'])
projected_expenses_df.tail()

,date,COGS,marketing,sg&a,shipping_loss,total
148,2023-12-27,3142.8,1125,NaN,293.996835,NaN
149,2023-12-28,2410.2,1125,NaN,225.776575,NaN
150,2023-12-29,2610.0,1125,NaN,243.643786,NaN
151,2023-12-30,2417.4,1125,NaN,225.776575,NaN
152,2023-12-31,2179.8,1125,NaN,203.036488,NaN


**SG&A**

Our SG&A can also be found in the local data directory. We project a constant salary of $30/hr capped at 40hrs/wk for both founders. There is also a perpetual monthly relocation expense paid to keep both founders in ATX.

In [19]:
# Pre-calculated
sga_df = pd.read_excel('../../data/imports/excel/sga_projected_expenses.xlsx', index_col=0)
projected_expenses_df['date'] = pd.to_datetime(projected_expenses_df['date'])
for row in sga_df.itertuples():
    date = pd.to_datetime(row.Index)
    # find this date in the date column of the projected_expenses_df and add the sga_amount
    projected_expenses_df.loc[projected_expenses_df['date'] == date, 'sg&a'] = row.sga_amount

projected_expenses_df['sg&a'] = projected_expenses_df['sg&a'].fillna(0)
projected_expenses_df.tail()

,date,COGS,marketing,sg&a,shipping_loss,total
148,2023-12-27,3142.8,1125,0,293.996835,NaN
149,2023-12-28,2410.2,1125,0,225.776575,NaN
150,2023-12-29,2610.0,1125,2400,243.643786,NaN
151,2023-12-30,2417.4,1125,0,225.776575,NaN
152,2023-12-31,2179.8,1125,0,203.036488,NaN


_________

In [20]:
projected_expenses_df['total'] = projected_expenses_df.sum(axis=1)
projected_expenses_df.tail()

/var/folders/rs/k8yvpht95zxbr8d7v79msjk00000gn/T/ipykernel_5651/1436562906.py:1: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.



,date,COGS,marketing,sg&a,shipping_loss,total
148,2023-12-27,3142.8,1125,0,293.996835,4561.796835
149,2023-12-28,2410.2,1125,0,225.776575,3760.976575
150,2023-12-29,2610.0,1125,2400,243.643786,6378.643786
151,2023-12-30,2417.4,1125,0,225.776575,3768.176575
152,2023-12-31,2179.8,1125,0,203.036488,3507.836488


Below you will see our projected daily expenses over time. COGS and Loss on Shipment of Goods appears noisy because they are driven by sales, which has been made purposefully noisy.

In [21]:
# Plot expense breakdown

expenses_fig = go.Figure(data=go.Scatter(x=projected_expenses_df['date'], y=projected_expenses_df['COGS'], name="Cost of Goods Sold"))
expenses_fig.add_trace(go.Scatter(x=projected_expenses_df['date'], y=projected_expenses_df['marketing'], name="Marketing Expenses"))
expenses_fig.add_trace(go.Scatter(x=projected_expenses_df['date'], y=projected_expenses_df['shipping_loss'], name="Loss on Shipment of Goods"))
expenses_fig.add_trace(go.Scatter(x=projected_expenses_df['date'], y=projected_expenses_df['total'], name="Total Expenses"))
expenses_fig.add_trace(go.Scatter(x=projected_expenses_df['date'], y=projected_expenses_df['sg&a'], name="SG&A"))
expenses_fig.update_layout(plot_bgcolor='#ece9f1')
expenses_fig.update_layout(title="Projected Daily Expenses", xaxis_title="Date", yaxis_title="Expenses")
expenses_fig.show()

After accounting for expenses, we can forecast cumulative PnL over time.

In [22]:
overall = sales_df['total_gross_sales'] - projected_expenses_df['total']
overall_rs = np.cumsum(overall)
overall_fig = go.Figure(data=go.Bar(x=projected_expenses_df['date'], y=overall, name="Daily PnL"))
overall_fig.update_layout(plot_bgcolor='#ece9f1')
overall_fig.add_trace(go.Scatter(x=projected_expenses_df['date'], y=overall_rs, name="Cumulative PnL"))
overall_fig.show()

We project a temporary unprofitable period during Q3 we undergo a period of elevated SG&A (compared to our historical near-zero levels) while both founders work full-time for the company. This period is quickly resolved as we begin to take advantage of untapped markets and bring adrafül to the masses. 

We also project a laddered decrease in CPU from the current level of 2.04 to a conservative estimate of 1.80 per unit, further increasing profitability. 

By EOY, we project $110-$130K EBITDA.

In [12]:
# export data

projected_expenses_df.to_excel('../../data/outputs/projected_daily_expenses.xlsx')